# **2.B FEATURE EXTRACTION**

## Dataset 2 - Phishing - 30% - 4437 urls

#### The objective of this notebook is to collect data and save it as a CSV file for Feature Extraction.

* Lexical Features
* Whois Features
* Popularity Features

#### This project is worked on Jupyter Notebook 

In [1]:
import pandas as pd
from urllib.parse import urlparse
import re
from bs4 import BeautifulSoup
import whois
import urllib.request
import time
import socket
from urllib.error import HTTPError
from datetime import datetime

In [2]:
#import dataset


legitimate_urls = pd.read_csv("/Users/jillkathleen/Desktop/Phishing-Analysis-Detection/Back-End/Data-Collection/Dataset2/D2_legit_label30.csv")



In [3]:
legitimate_urls

,url,label
0,www.hackcraft.net/raii/,good
1,promo.yahoo.com/user_research/,good
2,www.cpm.z80.de/small_c.html,good
3,www.codeguru.com/Cpp/G-M/opengl/,good
4,www.adaptivepath.com/publications/essays/archi...,good
...,...,...
4432,www.ccs.neu.edu/research/demeter/,good
4433,www.w3.org/Robot/,good
4434,webclipart.about.com/od/animalclipartlinks/l/b...,good
4435,www.march-hare.com/library/html/ud6blurb.htm,good


## 2.1 Lexical Features

* URL Length 
* URL Shortening Services “TinyURL”
* URL Presence of "@" Symbol
* URL Presence of special characters : _ ? = & etc
* URL Suspicious words (security sensitive words)
* URL Digit Count
* URL Protocol Count (http / https)
* URL Dot Count
* URL Hyphen Count
* Domain presence of IP Address
* Domain presence of hyphen / prefix or Suffix
* Sub Domain and Multi Sub Domains Count
* Redirecting "//" in URL (// position)
* URL presence of EXE


In [4]:
#class FeatureExtraction:
#    def __init__(url):
#        pass

# 1.Extracts domain from the given URL
def getDomain(url):
    domain = urlparse(url).netloc
    if re.match(r"^www.",domain):
        domain = domain.replace("www.","")
    return domain
    
# 2.Checks for IP address in URL (Have_IP)
def ip_address(url):
    try:
        ipaddress.ip_address(url)
        ip = 1
    except:
        ip = 0
    return ip
    
# 3.Checks the presence of @ in URL (Have_At)
def have_at_symbol(url):
    if "@" in url:
        at = 1 
    else:
        at = 0   
    return at
    
# 4.Finding the length of URL and categorizing (URL_Length)
def long_url(url):
    if len(url) < 54:
        length = 0    
    else:
        length = 1    
    return length

# 5.Gives number of '/' in URL (URL_Depth)
def getDepth(url):
    s = urlparse(url).path.split('/')
    depth = 0
    for j in range(len(s)):
        if len(s[j]) != 0:
            depth = depth+1
    return depth
        
# 6.Checking for redirection '//' in the url (Redirection)
def redirection(url):
    pos = url.rfind('//')
    if pos > 6:
        if pos > 7:
            return 1
        else:
            return 0
    else:
        return 0
    
# 7.Existence of “HTTPS” Token in the Domain Part of the URL (https_Domain)
def httpDomain(url):
    domain = urlparse(url).netloc
    if 'https://|http://' in domain:
        return 1
    else:
        return 0

    
# 8. Checking for Shortening Services in URL (Tiny_URL) 
def shortening_service(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                    'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                    'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                    'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                    'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                    'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                    'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                    'tr\.im|link\.zip\.net', url)
    if match:
        return 1               # phishing
    else:
        return 0               # legitimate
    
    
    
    
# 9.Checking for Prefix or Suffix Separated by (-) in the Domain (Prefix/Suffix)     
def prefix_suffix_separation(url):
    if "-" in urlparse(url).netloc:
        return 1            # phishing
    else:
        return 0            # legitimate
    
# 10. DNS Record 

    
# 11.Web traffic (Web_Traffic)
def web_traffic(url):
    try:
        url = urllib.parse.quote(url)
        rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find(
        "REACH")['RANK']
        rank = int(rank)
    except TypeError:
        return 1
    if rank <100000:
        return 1
    else:
        return 0
        
# 12.Survival time of domain: The difference between termination time and creation time (Domain_Age)  
def domainAge(domain_name):
    creation_date = domain_name.creation_date
    expiration_date = domain_name.expiration_date
    if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
        try:
            creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
            expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
        except:
            return 1
    if ((expiration_date is None) or (creation_date is None)):
        return 1
    elif ((type(expiration_date) is list) or (type(creation_date) is list)):
        return 1
    else:
        ageofdomain = abs((expiration_date - creation_date).days)
        if ((ageofdomain/30) < 6):
            age = 1
        else:
            age = 0
    return age

# 13.End time of domain: The difference between termination time and current time (Domain_End) 
def domainEnd(domain_name):
    expiration_date = domain_name.expiration_date
    if isinstance(expiration_date,str):
        try:
            expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
        except:
            return 1
    if (expiration_date is None):
        return 1
    elif (type(expiration_date) is list):
        return 1
    else:
        today = datetime.now()
        end = abs((expiration_date - today).days)
    if ((end/30) < 6):
        end = 0
    else:
        end = 1
    return end

# 14. Dot count
def dot_count(url):
    if url.count(".") < 3:
        return 0            # legitimate
    elif url.count(".") == 3:
        return 1            # suspicious
    else:
        return 1            # phishing
        
    
# 14. Special characters count
def specialcharCount(url):
    cnt = 0
    special_characters = [';','+=','_','?','=','&','[',']','/',':']
    for each_letter in url:
        if each_letter in special_characters:
            cnt = cnt + 1
    return cnt


# 15. 
def subdomCount(url):

    # separate protocol and domain then count the number of dots in domain
    
    domain = url.split("//")[-1].split("/")[0].split("www.")[-1]
    if(domain.count('.')<=1):
        return 0
    else:
        return 1

In [5]:
#Function to extract features
def featureExtraction(url,label):
    
    features = []
    
    features.append(getDomain(url))
    features.append(ip_address(url))
    features.append(have_at_symbol(url))
    features.append(long_url(url))
    features.append(getDepth(url))
    features.append(redirection(url))
    features.append(httpDomain(url))
    features.append(shortening_service(url))
    features.append(prefix_suffix_separation(url))
  
    dns = 0
    try:
        domain_name = whois.whois(urlparse(url).netloc)
    except:
        dns = 1
        
    features.append(dns)
    features.append(web_traffic(url))
    features.append(1 if dns == 1 else domainAge(domain_name))
    features.append(1 if dns == 1 else domainEnd(domain_name))
    
    features.append(dot_count(url))
    features.append(specialcharCount(url))
    features.append(subdomCount(url))
    

    features.append(label)
    
    
    return features

In [6]:
feature_names = ['domain', 'ip_present', 'at_present', 'url_length', 'url_depth','redirection', 
                      'https_domain', 'short_url', 'prefix/suffix', 'dns_record', 'web_traffic', 
                      'domain_age', 'domain_end', 'dot_count', 'specialchar_count','subdom_count', 'label']

label = 0

In [7]:
# Extracting the features & storing them in a list
# Lexical Features

# starting time
start_time = time.time()
print('\n')
print('Begin feature extraction for benign dataset.... \n')

##===================================##


#Extracting the feautres & storing them in a list
legit_features = []
rows = len(legitimate_urls['url'])
label = 0

for i in range(0, rows):
    url = legitimate_urls['url'][i]
    print(i), print(url)
    
    
    legit_features.append(featureExtraction(url,label))

    
##===================================##

elapsed = time.strftime("%H:%M:%S", time.gmtime(time.time() - start_time))
print('\n')
print(f"Runtime: Feature Extraction for legitimate dataset took:  {elapsed}")


print('\n\n\n\n')
print("***Legitimate Features")




Begin feature extraction for benign dataset.... 

0
www.hackcraft.net/raii/
1
promo.yahoo.com/user_research/
2
www.cpm.z80.de/small_c.html
3
www.codeguru.com/Cpp/G-M/opengl/
4
www.adaptivepath.com/publications/essays/archives/000328.php
5
www.osnews.com/story/5602
6
www.angelfire.com/trek/scottmaurer/
7
lyonsdenvideo.com/LDV.htm
8
www.nada.kth.se/~viggo/problemlist/compendium.html
9
en.wikibooks.org/wiki/Programming:Prolog
10
pourpre.com/chromograf/en/
11
www.cs.umd.edu/hcil/photolib/
12
www.digital-web.com/articles/introduction_to_xml/
13
tools.ietf.org/html/rfc1157
14
tools.ietf.org/html/rfc2468
15
www.ibm.com/developerworks/web/library/wa-seo2/
16
www.census.gov/foreign-trade
17
vmware0.tripod.com/index.htm
18
tools.ietf.org/html/rfc2699
19
www.zib.de/weiser/vtl/
20
www.global-home.org/convee/
21
www.cs.cmu.edu/~illah/
22
www.sanfords.net/Spots_free_graphics/Space_Background_Tutorial.htm
23
cs.anu.edu.au/wicapuc/
24
www.xml.com/resourceguide/
25
cernlib.web.cern.ch/cernlib/
26
www

206
www.algotech.dk/word-html-cleaner-input.htm
207
dwhome.dataway.ch/support/traceroute.aqua
208
home.earthlink.net/~joseph-ja/programs.html#Tcl-Tk
209
exophase.com/psp/basilisk-ii-for-psp-mac-os-emulator-6604.htm
210
sais.aisnet.org/sais2005/cfp.htm
211
tools.ietf.org/html/rfc2776
212
nikonimaging.com/global/products/scanner/index.htm
213
www.cs.brown.edu/research/plt/software/divascheme/
214
tools.ietf.org/html/rfc2769
215
www.mailutilities.com/adr/
216
www.hatz.ch/arkeroid/
217
slashdot.org/articles/older/980613118210_F.shtml
218
www.sankhya.com/info/varadhi.html
219
xml.coverpages.org/ace-overview.html
220
www.rajivshah.com/Case_Studies/Cookies/CookiesLinks.html
221
www.wingfoot.com/products.html
222
www.lysator.liu.se/~zap/ldb/
223
www.angelfire.com/games/UoLS/index.html
224
www.informatik.uni-kiel.de/~mh/FLP/
225
www.spec.org/gpc/opc.static/overview.html
226
www.sics.se/people/fredriko
227
www.examsguide.com/scjp/freesimulator.html
228
www.devx.com/DevX/Door/20763
229
www.bejo.c

412
tools.ietf.org/html/rfc1357
413
www.kaon.com/software/swmeson.html
414
www.webring.com/hub?ring=wiznetsitering
415
www.salesforce.com/platform/
416
www.musicent.com/index.html
417
www.codesandciphers.org.uk/lorenz/index.htm
418
tools.ietf.org/html/rfc3167
419
www.legacy.com/obituaries/news-leader/obituary-preview.aspx?n=heather-grootens-thompson&amp;pid=151715903
420
sourceforge.net/projects/ici/
421
support.microsoft.com/default.aspx?scid=support.microsoft.com:80/support/kb/articles/q169/1/73.asp&amp;NoWebContent=1
422
hitmill.com/html/index.html
423
www.cs.cmu.edu/~bam/
424
www.redflymarketing.com/blog/
425
vidar.gimp.org/gimp/
426
barbados.org/chess/
427
www.custard.org/\%7Eandrew/optimize.php
428
listserv.uga.edu/archives/sas-l.html
429
www.aiai.ed.ac.uk/~jeff/lisp/eulisp.html
430
www.sheepyvalley.com/sheep.htm
431
www.codeguru.com/Cpp
432
bricksandbeams.ecrater.com/rss-cat.php?cid=570291
433
www.angelfire.com/electronic/webdesigner/
434
tools.ietf.org/html/rfc2964
435
sites.go

606
fare.tunes.org/LispM.html
607
www.grcdi.nl/links.htm
608
www.gleamtech.com/products/wexlite.asp
609
tools.ietf.org/html/rfc3901
610
www.squid-cache.org/mail-archive/squid-users/hypermail.html
611
www.stonebroom.com/suppxp.htm
612
www.mhonarc.org/~ehood/MIME/MIME.html
613
tools.ietf.org/html/rfc2269
614
math.nist.gov/jazznet/
615
tools.ietf.org/html/rfc2573
616
www.mat.univie.ac.at/~neum/glopt.html
617
tools.ietf.org/html/rfc1644
618
tools.ietf.org/html/rfc2279
619
www.sweetsoftware.co.nz/lua_overview.html
620
www.automatedqa.com/products/aqnet.asp
621
elouai.com/doll-makers.php
622
www.moshesipper.com/pcm/
623
planeta.clix.pt/dominio/01-xuk.htm
624
tools.ietf.org/html/rfc3036
625
www.rpg.net/realm/cyber/
626
tools.ietf.org/html/rfc2071
627
www.dmoz.org/Bookmarks/H/herbridg/
628
news.bbc.co.uk/1/hi/technology/7561943.stm
629
sites.google.com/site/cottonlinters/
630
www.vitanuova.com/inferno/licence.html
631
dreamers.com/jspain/engmain.html
632
www.jguru.com/faq/JDBC/
633
whiteknuckl

803
homepage2.nifty.com/sakazuki/rde_en/
804
exodus.physics.ucla.edu/Fortran95/PSTIResearchLecSeries1.html
805
news.bbc.co.uk/2/hi/technology/2688619.stm
806
smithelec.theshoppe.com/welcmepage.htm
807
botik.ru/pub/local/scp/refal5/
808
projects.gnome.org/gtkglext/
809
www.badgers-in-foil.co.uk/projects/jactionscript/
810
www.gerd-tentler.de/tools/mimemail_perl/
811
developer.apple.com/java/javawebstart/
812
www.arsvcs.demon.co.uk/leisure/abuse/abuse.html
813
www.ecommercetimes.com/perl/story/13397.html
814
www.sis.pitt.edu/~mbsclass/hall_of_fame/
815
kuoi.com/~kamikaze/doc/minsky.html
816
www.ils-international.com/goldmine/cgmhome.htm
817
tools.ietf.org/html/rfc1077
818
www.angelfire.com/scifi/phaserfire/
819
www.leadershipchallenge.com/WileyCDA/
820
www.angelfire.com/country/bco/
821
www.win.tue.nl/~engels/go/index_en.html
822
www.ibm.com/software/awdtools/rmc/
823
www.hadrons.org/~guillem/debian/debtakeover/README
824
www.javacoffeebreak.com/books/extracts/jini/jini.html
825
dillygre

...

3501
www.tropical-music.com/indexgo.html
3502
www.tek-tips.com/threadminder.cfm?pid=195
3503
homepages.uc.edu/~hollisgf/nakedmud.html
3504
singapore_aberdeen.tripod.com/mackenzie.html
3505
artist4hire.net/Image.html
3506
www.debian.org/ports/m68k/
3507
news.bbc.co.uk/1/hi/entertainment/showbiz/2713133.stm
3508
clans.gameclubcentral.com/shoot/
3509
tools.ietf.org/html/rfc1595
3510
formale-sprachen.informatik.uni-oldenburg.de/persons/karl-heinz.pennemann/
3511
www.redbooks.ibm.com/abstracts/sg244986.html
3512
sourceforge.net/projects/rxvt
3513
www.angelfire.com/in2/wolfsong/index.html
3514
tools.ietf.org/html/rfc1206
3515
assignments.uspto.gov/assignments/q?db=pat&amp;pat=6058214
3516
www.iana.org/domains/root/db/
3517
pw.vsb.bc.ca/trek/
3518
jude.change-vision.com/jude-web/index.html
3519
www.cours.polymtl.ca/roboop/
3520
www.angelfire.com/fl5/cfarmsllc/
3521
www.jitbit.com/rssfeedcreator.aspx
3522
may.cs.ucla.edu/projects/maisie/
3523
tools.ietf.org/html/rfc1783
3524
www.dejavusoftware

3692
groups.yahoo.com/group/PCGF/
3693
www.zieglersoft.dk/public/zieglercollection.asp
3694
www3.sympatico.ca/craig.kadoke/true/
3695
www.storefront.com/photo_kiosk.html
3696
library.gnome.org/devel/gtk/
3697
www.freescripts.com/ASP/scripts.html
3698
tools.ietf.org/html/rfc338
3699
anfi.homeunix.net/sendmail/
3700
www.k5n.us/Ilib.php
3701
static.userland.com/userLandDiscussArchive/msg022178.html
3702
faculty.washington.edu/wcalvin/
3703
www.westwindcos.com/familyhistory/
3704
slashdot.org/developers/01/07/08/1955209.shtml
3705
www.oswego.edu/academics/colleges_and_departments/departments/computer_science/
3706
search.cpan.org/~nkh/
3707
www.pandia.com/sw-2001/44-zeal.html
3708
tools.ietf.org/html/rfc1777
3709
www.21stcentury.co.uk/robotics/
3710
home.neopets.com/templates/homepage.phtml?pet_name=Hidden_Dream
3711
www.bitsavers.org/pdf/ibm/360/pls/GC28-6794-0_PLSIIguideMay74.pdf
3712
www.research.att.com/~bs/bs_faq2.html
3713
www.hollandnumerics.demon.co.uk/ROYSTON.HTM
3714
hem.passagen

3889
en.wikipedia.org/wiki/Programming_style
3890
www.javaworld.com/javaworld/jw-02-1999/jw-02-toolbox_p.html
3891
www.developer.com/net/vb/article.php/1540311
3892
www.marinelistingservice.com/colorblindtest/
3893
www.aspupload.com/aspjpeg.html
3894
www.apacheweek.com/features/dynamicpages
3895
www.m-kagaku.co.jp/index_en.htm
3896
www.pearsonhighered.com/educator/academic/product/0,,0201136880,00\%2Ben-USS_01DBC.html
3897
www.zdnet.com/news/netscape-acquires-newhoo/101006
3898
www.csse.monash.edu.au/~debbiep/billabong/rexpress.html
3899
tools.ietf.org/html/rfc3069
3900
www.nanotech-now.com/nanotechnology-glossary-A-C.htm
3901
www.cchem.berkeley.edu/jsngrp/fortran.html
3902
www.webterrace.com/graphics/index.htm
3903
tools.ietf.org/html/rfc2998
3904
atrey.karlin.mff.cuni.cz/~vojtech/joystick/
3905
www.ics.uci.edu/~franz/
3906
kernel.org/pub/linux/kernel/people/paulmck/perfbook/perfbook.html
3907
www.shimonsandler.com/web-directories-interview-with-greg-hartnett/
3908
www.apromotionguide

4074
www.cs.rpi.edu/~musser/stl-bigger-example.html
4075
www.angelfire.com/oh4/810/index.html
4076
grc.com/su-firewalls.htm
4077
www.sybase.com/support/techdocs/
4078
www.pctools.com/forum/
4079
www.pearsonhighered.com/educator/academic/product/0,,0130260363,00\%2Ben-USS_01DBC.html
4080
www.serpik.com/alaunch/
4081
www.renesas.com/products/mpumcu/superh/superh_landing.jsp
4082
tech.groups.yahoo.com/group/wsdl/
4083
tools.ietf.org/html/rfc652
4084
ps2.gamespy.com/playstation-2/reservoir-dogs/
4085
iptables-tutorial.frozentux.net/iptables-tutorial.html
4086
people.cs.kuleuven.be/~dirk.craeynest/ada-belgium/success/success.html
4087
www.royal.gov.uk/LatestNewsandDiary/Pressreleases/2002/50factsaboutTheDukeofEdinburgh.aspx
4088
web.me.com/hiangle/
4089
tools.ietf.org/html/rfc1778
4090
www.kirchgessner.net/gimp.html
4091
www.alhem.net/Sockets/
4092
damiendebin.net/archives/zzip/
4093
tools.ietf.org/html/rfc1915
4094
www.lrv.fri.uni-lj.si/facedb.html
4095
www.magellass.com/prod-wb.html
4096


4265
www.gcn.com/online/vol1_no1/22933-1.html?topic=daily-updates
4266
www.kinesis-ergo.com/advantage.htm
4267
members.tripod.com/fbaryol/homepage.html
4268
www.redbrick.dcu.ie/~cortex/
4269
tools.ietf.org/html/rfc1873
4270
www.powerquest.com/partitionmagic/index.html
4271
tools.ietf.org/html/rfc2685
4272
pub.cozmixng.org/~the-rwiki/
4273
www.ddj.com/architect/184415220/
4274
www.opencrypt.com/?ab01
4275
budgetvideo.homestead.com/mainpage.html
4276
www.eltima.com/products/showkit2/
4277
tools.ietf.org/html/rfc2001
4278
web.singnet.com.sg/~cbcpl/
4279
www.oreilly.com/catalog/9780596527310/
4280
www.caliban.org/ruby/ruby-google.shtml
4281
tools.ietf.org/html/rfc2057
4282
my.execpc.com/~absinthe/design.htm
4283
members.tripod.com/gsraj/misc/ejbmts/ejbmtscomp.html
4284
www.users.on.net/~geosurveys/
4285
www.helsinki.fi/iehc2006/papers2/Thomson.pdf
4286
www.cse.unsw.edu.au/~lxue/
4287
www.ssi-developer.net/main/templates/
4288
merlot.stat.uconn.edu/~nalini/programs.html
4289
tools.ietf.org/

In [10]:
#Converting the list to dataframe

legitimate = pd.DataFrame(legit_features, columns= feature_names)
legitimate.head()

,domain,ip_present,at_present,url_length,url_depth,redirection,https_domain,short_url,prefix/suffix,dns_record,web_traffic,domain_age,domain_end,dot_count,specialchar_count,subdom_count,label
0,,0,0,0,2,0,0,0,0,0,1,1,1,0,2,0,0
1,,0,0,0,2,0,0,0,0,0,1,1,1,0,3,1,0
2,,0,0,0,2,0,0,0,0,0,1,1,1,1,2,1,0
3,,0,0,0,4,0,0,0,0,0,1,1,1,0,4,0,0
4,,0,0,1,5,0,0,0,0,0,0,1,1,1,4,0,0


In [13]:
# Storing the extracted legitimate URLs fatures to csv file

legitimate.to_csv('/Users/jillkathleen/Desktop/Phishing-Analysis-Detection/Back-End/Extracted CSVs/from FeatureExtraction/Data-2/Data-2-benign-extracted.csv', index= False)

